# Module 08: Deployment Strategies

**Duration:** 75-90 minutes  
**Difficulty:** Advanced  
**Prerequisites:** Module 07

## Learning Objectives

1. Deploy n8n to cloud platforms
2. Configure production environment
3. Set up HTTPS and SSL
4. Implement backup strategies
5. Monitor and scale

## Deployment Options

### 1. DigitalOcean (Recommended for beginners)

**Cost:** $6-12/month for basic droplet

**Steps:**
1. Create droplet (Ubuntu 22.04)
2. Install Docker
3. Deploy with docker-compose
4. Configure domain
5. Setup SSL with Let's Encrypt

### 2. AWS (Amazon Web Services)

**Options:**
- EC2 instance
- ECS (Elastic Container Service)
- Lightsail (simplified)

### 3. Railway (Easiest)

**Features:**
- One-click deploy
- Automatic SSL
- Built-in monitoring
- Cost: ~$5-15/month

## Production Docker Compose

```yaml
version: '3.8'

services:
  n8n:
    image: n8nio/n8n:latest
    restart: always
    ports:
      - "5678:5678"
    environment:
      - N8N_HOST=your-domain.com
      - N8N_PROTOCOL=https
      - WEBHOOK_URL=https://your-domain.com/
      
      # Database (PostgreSQL recommended for production)
      - DB_TYPE=postgresdb
      - DB_POSTGRESDB_HOST=postgres
      - DB_POSTGRESDB_DATABASE=n8n
      - DB_POSTGRESDB_USER=n8n
      - DB_POSTGRESDB_PASSWORD=${DB_PASSWORD}
      
      # Encryption key (generate secure key!)
      - N8N_ENCRYPTION_KEY=${ENCRYPTION_KEY}
      
    volumes:
      - n8n_data:/home/node/.n8n
    depends_on:
      - postgres

  postgres:
    image: postgres:15-alpine
    restart: always
    environment:
      - POSTGRES_DB=n8n
      - POSTGRES_USER=n8n
      - POSTGRES_PASSWORD=${DB_PASSWORD}
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  n8n_data:
  postgres_data:
```

## SSL/HTTPS Setup

### Using nginx-proxy + Let's Encrypt

```yaml
services:
  nginx-proxy:
    image: nginxproxy/nginx-proxy
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - /var/run/docker.sock:/tmp/docker.sock:ro
      - certs:/etc/nginx/certs
      - vhost:/etc/nginx/vhost.d
      - html:/usr/share/nginx/html

  letsencrypt:
    image: nginxproxy/acme-companion
    volumes:
      - /var/run/docker.sock:/var/run/docker.sock:ro
      - certs:/etc/nginx/certs
      - vhost:/etc/nginx/vhost.d
      - html:/usr/share/nginx/html
    environment:
      - DEFAULT_EMAIL=your-email@example.com

  n8n:
    environment:
      - VIRTUAL_HOST=n8n.your-domain.com
      - LETSENCRYPT_HOST=n8n.your-domain.com
      - LETSENCRYPT_EMAIL=your-email@example.com
```

## Backup Strategy

### 1. Database Backups

In [ ]:
# Create backup script
backup_script = '''
#!/bin/bash

# Backup configuration
BACKUP_DIR="/backups/n8n"
DATE=$(date +%Y%m%d_%H%M%S)

# Create backup directory
mkdir -p $BACKUP_DIR

# Backup PostgreSQL database
docker exec n8n_postgres pg_dump -U n8n n8n > $BACKUP_DIR/n8n_db_$DATE.sql

# Backup n8n data
docker run --rm -v n8n_data:/data -v $BACKUP_DIR:/backup \\
  alpine tar czf /backup/n8n_data_$DATE.tar.gz -C /data .

# Keep only last 30 days
find $BACKUP_DIR -name "*.sql" -mtime +30 -delete
find $BACKUP_DIR -name "*.tar.gz" -mtime +30 -delete

echo "Backup completed: $DATE"
'''

print("Backup script created")
print("Save as: backup_n8n.sh")
print("Make executable: chmod +x backup_n8n.sh")
print("Add to crontab: 0 2 * * * /path/to/backup_n8n.sh")

### 2. Workflow Exports (Python)

In [ ]:
import sys
import os
from datetime import datetime
sys.path.append('../scripts')
from n8n_client import N8nClient

def backup_all_workflows(client, backup_dir='../data/exports'):
    """
    Export all workflows to JSON files
    """
    # Create backup directory with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_path = os.path.join(backup_dir, timestamp)
    os.makedirs(backup_path, exist_ok=True)
    
    # Get all workflows
    workflows = client.list_workflows()
    
    print(f"Backing up {len(workflows)} workflows...\n")
    
    for wf in workflows:
        # Safe filename
        filename = f"{wf['id']}_{wf['name'].replace(' ', '_')}.json"
        filepath = os.path.join(backup_path, filename)
        
        try:
            client.export_workflow(wf['id'], filepath)
            print(f"✅ Exported: {wf['name']}")
        except Exception as e:
            print(f"❌ Failed: {wf['name']} - {e}")
    
    print(f"\nBackup completed: {backup_path}")
    return backup_path

# Example usage
# client = N8nClient(...)
# backup_all_workflows(client)

## Monitoring

### Health Checks

In [ ]:
# Simple health check script
import requests
import time

def monitor_n8n_health(base_url="http://localhost:5678", interval=60):
    """
    Monitor n8n health status
    """
    print(f"Monitoring n8n at {base_url}...\n")
    
    while True:
        try:
            response = requests.get(f"{base_url}/healthz", timeout=5)
            
            if response.status_code == 200:
                print(f"✅ {time.strftime('%Y-%m-%d %H:%M:%S')} - Healthy")
            else:
                print(f"⚠️  {time.strftime('%Y-%m-%d %H:%M:%S')} - Status: {response.status_code}")
                
        except requests.exceptions.RequestException as e:
            print(f"❌ {time.strftime('%Y-%m-%d %H:%M:%S')} - Error: {e}")
        
        time.sleep(interval)

# Run for 5 minutes as example
# monitor_n8n_health(interval=10)

## Summary

**Learned:**
- Cloud deployment options
- Production configuration
- SSL/HTTPS setup
- Backup strategies
- Monitoring approaches

**Next:** Module 09 - Advanced Projects